In [1]:
import torch
from torch import Tensor
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

import dlc_bci as bci
from helpers import *

torch.manual_seed(1)

%load_ext autoreload
%autoreload 2

### Importing the data

In [2]:
# Subset of data sampled at 100Hz
train_input , train_target = bci.load(root = './data_bci')
print("Train input :", str(type(train_input)), train_input.size()) 
print("Train target :", str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print("Test input :", str(type(test_input)), test_input.size()) 
print("Test target :", str(type(test_target)), test_target.size())

Train input : <class 'torch.FloatTensor'> torch.Size([316, 28, 50])
Train target : <class 'torch.LongTensor'> torch.Size([316])
Test input : <class 'torch.FloatTensor'> torch.Size([100, 28, 50])
Test target : <class 'torch.LongTensor'> torch.Size([100])


In [32]:
# Full data sampled at 1Khz
train_input , train_target = bci.load(root = './data_bci', one_khz = True)
print("Train input :", str(type(train_input)), train_input.size()) 
print("Train target :", str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False, one_khz = True)
print("Test input :", str(type(test_input)), test_input.size()) 
print("Test target :", str(type(test_target)), test_target.size())

Train input : <class 'torch.FloatTensor'> torch.Size([316, 28, 500])
Train target : <class 'torch.LongTensor'> torch.Size([316])
Test input : <class 'torch.FloatTensor'> torch.Size([100, 28, 500])
Test target : <class 'torch.LongTensor'> torch.Size([100])


In [33]:
train_input[0]


  30.8000   33.1000   37.9000  ...    76.9000   75.9000   72.3000
 -22.6000  -19.2000  -11.8000  ...    26.8000   22.9000   16.4000
  11.2000   16.7000   26.4000  ...    64.1000   62.0000   55.9000
             ...                ⋱                ...             
   0.5000    0.5000    0.4000  ...    45.1000   46.2000   46.6000
  11.0000   10.6000    9.8000  ...    41.7000   41.5000   40.9000
  -9.9000  -10.0000  -10.3000  ...    32.9000   33.8000   32.8000
[torch.FloatTensor of size 28x500]

In [34]:
train_target


 0
 1
 0
 1
 0
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 1
 0
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 1
 1
 0
 1
 1
 0
 0
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 1
 1
 0
 0
 1
 1
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 0
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 1
 0
 0
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 0
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 0
 0
 1
 1
 0
 1
 1
 0
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 316]

In [5]:
# normalize mean to 0
train_input.sub_(train_input.mean())
test_input.sub_(test_input.mean())

# normalize variance to 1
train_input.div_(train_input.std())
test_input.div_(test_input.std())

print('done')

done


In [59]:
#total = train_input.new(3160,28,50)
seq = []
for i in range(10):
    seq.append(train_input[:,:,i::10])
total = torch.cat(seq)'

In [60]:
total


( 0  ,.,.) = 
   30.8000   45.4000   28.2000  ...    81.2000   59.7000   78.5000
  -22.6000   -0.7000  -24.7000  ...    31.5000    2.1000   34.2000
   11.2000   45.5000    9.7000  ...    79.5000   36.1000   86.4000
              ...                ⋱                ...             
    0.5000    4.2000    9.2000  ...    38.6000   38.4000   36.1000
   11.0000   18.1000   31.3000  ...    53.6000   53.6000   43.7000
   -9.9000   -1.3000    7.3000  ...    37.0000   37.7000   28.8000

( 1  ,.,.) = 
  135.7000  134.8000  145.7000  ...    77.8000   91.4000   85.1000
   16.9000    0.7000   19.3000  ...   -40.3000  -12.2000  -33.5000
   23.9000    9.1000   23.7000  ...   -17.9000    7.1000  -13.2000
              ...                ⋱                ...             
  -18.8000  -16.7000  -28.1000  ...   -30.9000  -20.1000  -15.5000
    6.1000    5.8000   -4.8000  ...    -6.7000    9.6000   19.0000
  -27.5000  -18.5000  -30.1000  ...   -44.0000  -32.3000  -17.8000

( 2  ,.,.) = 
   69.4000   57.0

In [63]:
train_input[1,:,::10]


 135.7000  134.8000  145.7000  ...    77.8000   91.4000   85.1000
  16.9000    0.7000   19.3000  ...   -40.3000  -12.2000  -33.5000
  23.9000    9.1000   23.7000  ...   -17.9000    7.1000  -13.2000
             ...                ⋱                ...             
 -18.8000  -16.7000  -28.1000  ...   -30.9000  -20.1000  -15.5000
   6.1000    5.8000   -4.8000  ...    -6.7000    9.6000   19.0000
 -27.5000  -18.5000  -30.1000  ...   -44.0000  -32.3000  -17.8000
[torch.FloatTensor of size 28x50]

In [48]:
train_input[0,:,6]


  41.8000
 -10.3000
  38.1000
  27.9000
  13.2000
   3.9000
   3.2000
  -5.8000
  49.0000
  59.0000
 101.8000
 -17.1000
  -9.1000
  -3.5000
  16.9000
  13.9000
  -0.9000
  -4.6000
   0.4000
  36.9000
  10.1000
  24.5000
  34.6000
  63.4000
  30.3000
   7.9000
  19.2000
  -0.4000
[torch.FloatTensor of size 28]